# Stock NeurIPS2018 Part 1. Data
This series is a reproduction of paper *the process in the paper Practical Deep Reinforcement Learning Approach for Stock Trading*. 

This is the first part of the NeurIPS2018 series, introducing how to use FinRL to fetch and process data that we need for ML/RL trading.

Other demos can be found at the repo of [FinRL-Tutorials]((https://github.com/AI4Finance-Foundation/FinRL-Tutorials)).

# Part 1. Install Packages

In [59]:
## install finrl library
#!pip install git+https://github.com/AI4Finance-Foundation/FinRL.git

In [60]:
import pandas as pd
import numpy as np
import datetime
import yfinance as yf

from finrl.meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.meta.preprocessor.preprocessors import FeatureEngineer, data_split
from finrl import config_tickers
from finrl.config import INDICATORS

import itertools

# Part 2. Fetch data

[yfinance](https://github.com/ranaroussi/yfinance) is an open-source library that provides APIs fetching historical data form Yahoo Finance. In FinRL, we have a class called [YahooDownloader](https://github.com/AI4Finance-Foundation/FinRL/blob/master/finrl/meta/preprocessor/yahoodownloader.py) that use yfinance to fetch data from Yahoo Finance.

**OHLCV**: Data downloaded are in the form of OHLCV, corresponding to **open, high, low, close, volume,** respectively. OHLCV is important because they contain most of numerical information of a stock in time series. From OHLCV, traders can get further judgement and prediction like the momentum, people's interest, market trends, etc.

## Data for a single ticker

Here we provide two ways to fetch data with single ticker, let's take Apple Inc. (AAPL) as an example.

### Using yfinance

In [61]:
aapl_df_yf = yf.download(tickers = "aapl", start='2020-01-01', end='2020-01-31')

[*********************100%***********************]  1 of 1 completed


In [62]:
aapl_df_yf.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2020-01-02,74.059998,75.150002,73.797501,75.087502,72.876122,135480400
2020-01-03,74.287498,75.144997,74.125000,74.357498,72.167603,146322800
2020-01-06,73.447502,74.989998,73.187500,74.949997,72.742668,118387200
2020-01-07,74.959999,75.224998,74.370003,74.597504,72.400536,108872000
2020-01-08,74.290001,76.110001,74.290001,75.797501,73.565208,132079200


### Using FinRL

In FinRL's YahooDownloader, we modified the data frame to the form that convenient for further data processing process. We use adjusted close price instead of close price, and add a column representing the day of a week (0-4 corresponding to Monday-Friday).

In [63]:
aapl_df_finrl = YahooDownloader(start_date = '2020-01-01',
                                end_date = '2024-05-31',
                                ticker_list = ['aapl']).fetch_data()

[*********************100%***********************]  1 of 1 completed

Shape of DataFrame:  (1110, 8)


In [64]:
aapl_df_finrl.head()

,date,open,high,low,close,volume,tic,day
0,2020-01-02,74.059998,75.150002,73.797501,72.876114,135480400,aapl,3
1,2020-01-03,74.287498,75.144997,74.125000,72.167618,146322800,aapl,4
2,2020-01-06,73.447502,74.989998,73.187500,72.742661,118387200,aapl,0
3,2020-01-07,74.959999,75.224998,74.370003,72.400551,108872000,aapl,1
4,2020-01-08,74.290001,76.110001,74.290001,73.565201,132079200,aapl,2


## Data for the chosen tickers

In [65]:
config_tickers.DOW_30_TICKER

['AXP',
 'AMGN',
 'AAPL',
 'BA',
 'CAT',
 'CSCO',
 'CVX',
 'GS',
 'HD',
 'HON',
 'IBM',
 'INTC',
 'JNJ',
 'KO',
 'JPM',
 'MCD',
 'MMM',
 'MRK',
 'MSFT',
 'NKE',
 'PG',
 'TRV',
 'UNH',
 'CRM',
 'VZ',
 'V',
 'WBA',
 'WMT',
 'DIS',
 'DOW']

In [66]:
TRAIN_START_DATE = '2009-01-05'
TRAIN_END_DATE = '2023-07-01'
TRADE_START_DATE = '2023-07-01'
TRADE_END_DATE = '2024-09-14'

In [67]:
df_raw = YahooDownloader(start_date = TRAIN_START_DATE,
                     end_date = TRADE_END_DATE,
                     ticker_list = [
    'AOT.BK', 'ADVANC.BK', 'CPALL.BK', 'SCC.BK', 'PTT.BK',
    'KBANK.BK', 'TOP.BK', 'TRUE.BK', 'BTS.BK',
    'MBK.BK', 'BANPU.BK', 'IRPC.BK', 'SIRI.BK', 'TISCO.BK',
    'GULF.BK', 'LH.BK', 'CPF.BK', 'VGI.BK', 'RATCH.BK'
]).fetch_data()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

Shape of DataFrame:  (69572, 8)


In [68]:
df_raw.head()

,date,open,high,low,close,volume,tic,day
0,2009-01-05,80.000000,81.000000,77.500000,33.242226,10319500,ADVANC.BK,0
1,2009-01-05,1.850000,1.960000,1.840000,1.606348,31267000,AOT.BK,0
2,2009-01-05,15.146944,16.547083,15.019659,7.740382,123814816,BANPU.BK,0
3,2009-01-05,1.471585,1.530448,1.412722,0.811429,2409272,BTS.BK,0
4,2009-01-05,6.250000,6.300000,6.150000,4.381951,13774600,CPALL.BK,0


In [69]:
train = pd.read_csv('train_data.csv')
trade = pd.read_csv('trade_data.csv')

# Part 3: Preprocess Data
We need to check for missing data and do feature engineering to convert the data point into a state.
* **Adding technical indicators**. In practical trading, various information needs to be taken into account, such as historical prices, current holding shares, technical indicators, etc. Here, we demonstrate two trend-following technical indicators: MACD and RSI.
* **Adding turbulence index**. Risk-aversion reflects whether an investor prefers to protect the capital. It also influences one's trading strategy when facing different market volatility level. To control the risk in a worst-case scenario, such as financial crisis of 2007–2008, FinRL employs the turbulence index that measures extreme fluctuation of asset price.

Hear let's take **MACD** as an example. Moving average convergence/divergence (MACD) is one of the most commonly used indicator showing bull and bear market. Its calculation is based on EMA (Exponential Moving Average indicator, measuring trend direction over a period of time.)

In [70]:
fe = FeatureEngineer(use_technical_indicator=True,
                     tech_indicator_list = INDICATORS,
                     use_vix=True,
                     use_turbulence=True,
                     user_defined_feature = False)

processed = fe.preprocess_data(df_raw)

Successfully added technical indicators


[*********************100%***********************]  1 of 1 completed


Shape of DataFrame:  (3949, 8)
Successfully added vix
Successfully added turbulence index


In [71]:
list_ticker = processed["tic"].unique().tolist()
list_date = list(pd.date_range(processed['date'].min(),processed['date'].max()).astype(str))
combination = list(itertools.product(list_date,list_ticker))

processed_full = pd.DataFrame(combination,columns=["date","tic"]).merge(processed,on=["date","tic"],how="left")
processed_full = processed_full[processed_full['date'].isin(processed['date'])]
processed_full = processed_full.sort_values(['date','tic'])

processed_full = processed_full.fillna(0)

In [72]:
processed_full.head()

,date,tic,open,high,low,close,volume,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix,turbulence
0,2009-01-05,ADVANC.BK,80.000000,81.000000,77.500000,33.242226,10319500.0,0.0,0.0,33.626909,32.436763,0.0,-66.666667,100.0,33.242226,33.242226,39.080002,0.0
1,2009-01-05,AOT.BK,1.850000,1.960000,1.840000,1.606348,31267000.0,0.0,0.0,33.626909,32.436763,0.0,-66.666667,100.0,1.606348,1.606348,39.080002,0.0
2,2009-01-05,BANPU.BK,15.146944,16.547083,15.019659,7.740382,123814816.0,0.0,0.0,33.626909,32.436763,0.0,-66.666667,100.0,7.740382,7.740382,39.080002,0.0
3,2009-01-05,BTS.BK,1.471585,1.530448,1.412722,0.811429,2409272.0,0.0,0.0,33.626909,32.436763,0.0,-66.666667,100.0,0.811429,0.811429,39.080002,0.0
4,2009-01-05,CPALL.BK,6.250000,6.300000,6.150000,4.381951,13774600.0,0.0,0.0,33.626909,32.436763,0.0,-66.666667,100.0,4.381951,4.381951,39.080002,0.0


# Part 4: Save the Data

### Split the data for training and trading

In [73]:
print()

In [74]:
train = data_split(processed_full, TRAIN_START_DATE,TRAIN_END_DATE)
trade = data_split(processed_full, TRADE_START_DATE,TRADE_END_DATE)
print(len(train))
print(len(trade))

58072
4777


### Save data to csv file

For Colab users, you can open the virtual directory in colab and manually download the files.

For users running on your local environment, the csv files should be at the same directory of this notebook.

In [75]:
train.to_csv('train_data.csv')
trade.to_csv('trade_data.csv')